# 1. Import

In [ ]:
# 연산 처리를 위한 패키지
import numpy as np
import pandas as pd
from pandas import DataFrame
from math import sqrt

# 데이터 분석을 위한 패키지
import statsmodels.api as sm

# 시각화를 위한 패키지
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

# 필요모듈 import
import os
import openpyxl
from datetime import datetime
from tqdm import tqdm

# 그래프를 실제로 그리기 위한 설정
%matplotlib inline

# 머신러닝 패키지
from catboost import CatBoostRegressor
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression  # 1. Linear Regression
from sklearn.linear_model import Lasso  # 2. Lasso
from sklearn.linear_model import Ridge  # 3. Ridge
from xgboost.sklearn import XGBRegressor  # 4. XGBoost
from lightgbm.sklearn import LGBMRegressor  # 5. LightGBM

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import random
import optuna
from optuna.samplers import TPESampler
# KFold(CV), partial : optuna를 사용하기 위함
from sklearn.model_selection import KFold
from functools import partial

# 폰트 처리
# plt.rc('font', family='NanumGothic')        # for windows
plt.rc('font', family='AppleGothic') # For MacOS

import warnings
warnings.filterwarnings('ignore')

# 2. Functions

In [ ]:
# Function sets
# feature constructing
# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ#


#                 3-2-1. 고유번호 끊어주기                  #



# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ#


def setting_data(train, test, one, two, three, four, five, six):
  test['송하인_격자공간고유번호'] = test['송하인_격자공간고유번호'].astype(str)
  test['수하인_격자공간고유번호'] = test['수하인_격자공간고유번호'].astype(str)
  train['송하인_격자공간고유번호'] = train['송하인_격자공간고유번호'].astype(str)
  train['수하인_격자공간고유번호'] = train['수하인_격자공간고유번호'].astype(str)

  train['송하인_코드1'] = train['송하인_격자공간고유번호'].str.slice(int(one),int(two))
  train['송하인_코드2'] = train['송하인_격자공간고유번호'].str.slice(int(two),int(three))
  train['송하인_코드3'] = train['송하인_격자공간고유번호'].str.slice(int(three),int(four))
  train['송하인_코드4'] = train['송하인_격자공간고유번호'].str.slice(int(four),int(five))
  train['송하인_코드5'] = train['송하인_격자공간고유번호'].str.slice(int(five),int(six))

  train['수하인_코드1'] = train['수하인_격자공간고유번호'].str.slice(int(one),int(two))
  train['수하인_코드2'] = train['수하인_격자공간고유번호'].str.slice(int(two),int(three))
  train['수하인_코드3'] = train['수하인_격자공간고유번호'].str.slice(int(three),int(four))
  train['수하인_코드4'] = train['수하인_격자공간고유번호'].str.slice(int(four),int(five))
  train['수하인_코드5'] = train['수하인_격자공간고유번호'].str.slice(int(five),int(six))

  test['송하인_코드1'] = test['송하인_격자공간고유번호'].str.slice(int(one),int(two))
  test['송하인_코드2'] = test['송하인_격자공간고유번호'].str.slice(int(two),int(three))
  test['송하인_코드3'] = test['송하인_격자공간고유번호'].str.slice(int(three),int(four))
  test['송하인_코드4'] = test['송하인_격자공간고유번호'].str.slice(int(four),int(five))
  test['송하인_코드5'] = test['송하인_격자공간고유번호'].str.slice(int(five),int(six))

  test['수하인_코드1'] = test['수하인_격자공간고유번호'].str.slice(int(one),int(two))
  test['수하인_코드2'] = test['수하인_격자공간고유번호'].str.slice(int(two),int(three))
  test['수하인_코드3'] = test['수하인_격자공간고유번호'].str.slice(int(three),int(four))
  test['수하인_코드4'] = test['수하인_격자공간고유번호'].str.slice(int(four),int(five))
  test['수하인_코드5'] = test['수하인_격자공간고유번호'].str.slice(int(five),int(six))

  train = train[['물품_카테고리', '송하인_코드1','송하인_코드2','송하인_코드3', '송하인_코드4', '송하인_코드5','수하인_코드1', '수하인_코드2', '수하인_코드3', '수하인_코드4', '수하인_코드5','운송장_건수']]

  test = test[['물품_카테고리','송하인_코드1','송하인_코드2', '송하인_코드3', '송하인_코드4', '송하인_코드5','수하인_코드1', '수하인_코드2', '수하인_코드3', '수하인_코드4', '수하인_코드5',]]

  for col in test.columns:
    train[col]=train[col].astype('category')
    test[col]=test[col].astype('category')
                
  return train, test


# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ#


#                 3-2-2. 카테고리 대분류 추가               #



# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ#


def large_category (train, test):
    
    change_cat_dict = {
    ('아우터', '신발', '상의', '하의', '기타패션의류', '양말/스타킹', '잠옷', '원피스/점프슈트') : '의류',
    ('농산물') : '농산물',
    ('수납가구', '아웃도어가구', '카페트/러그', '서재/사무용가구', '주방가구', '침실가구') : '가구', 
    ('의료용품', '건강용품', '생활용품', '문구/사무용품', '주방용품', '침구세트', '침구단품', '인테리어소품', '공구', '커튼/블라인드', 'DIY자재/용품', '위생/건강용품', '뷰티소품', '세탁용품', '수납/정리용품', '반려동물', '완구/매트', '홈데코') :  '생활용품',
    ('수산', '음료', '기타식품', '건강식품', '가공식품', '축산', '과자', '다이어트식품', '냉동/간편조리식품', '김치', '반찬') : '수산/기타식품',
    ('음반', '문화컨텐츠') : '문화',
    ('스킨케어', '헤어케어', '바디케어', '클렌징', '기타화장품/미용', '헤어스타일링', '베이스메이크업', '향수', '선케어', '네일케어', '색조메이크업', '남성화장품', '구강위생용품', '눈관리용품', '욕실용품') : '미용',
    ('기저귀/물티슈', '분유/이유식/아기간식', '기타출산/육아', '유아가구', '출산/유아동잡화', '출산/유아동의류') : '영유아',
    ('기타디지털/가전', '모니터', 'PC', 'PC주변기기', '계절가전', '태블릿PC/노트북액세서리', '스마트디바이스', '스마트디바이스액세서리', '게임기/타이틀', '생활가전', '음향가전', '주방가전', '이미용가전') : '디지털가전',
    ('기타스포츠/레저', '등산', '낚시', '스포츠잡화', '캠핑', '헬스', '취미용품', '골프') : '스포츠',
    ('헤어액세서리', '모자', '기타패션잡화', '언더웨어', '패션소품', '선글라스/안경테', '지갑', '기능성', '가방', '헤어액세서리', '주얼리', '자동차용품', '재활운동용품') : '패션잡화',
    }   

    train['물품_카테고리_소분류'] = train['물품_카테고리']
    test['물품_카테고리_소분류'] = test['물품_카테고리']
    train['물품_카테고리_대분류'] = train['물품_카테고리']
    test['물품_카테고리_대분류'] = test['물품_카테고리']

    train = train.replace({'물품_카테고리_대분류': change_cat_dict})
    test = test.replace({'물품_카테고리_대분류': change_cat_dict})

    train = train[['물품_카테고리_대분류','물품_카테고리_소분류', '송하인_코드1','송하인_코드2','송하인_코드3', '송하인_코드4', '송하인_코드5','수하인_코드1', '수하인_코드2', '수하인_코드3', '수하인_코드4', '수하인_코드5','운송장_건수']]

    test = test[['물품_카테고리_대분류','물품_카테고리_소분류', '송하인_코드1','송하인_코드2', '송하인_코드3', '송하인_코드4', '송하인_코드5','수하인_코드1', '수하인_코드2', '수하인_코드3', '수하인_코드4', '수하인_코드5',]]
    
    for col in test.columns:
        train[col]=train[col].astype('category')
        test[col]=test[col].astype('category')
    
    return train, test



In [ ]:
# Encoders

# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ#


#                     3-3. 인코딩                        #



# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ#



# 원핫인코딩, 라벨인코딩 선택

## 원핫인코딩
def one_hot_encoder(train, test):
  categorical_features = train.columns[0:12]
  temp = train.columns[13:]
  card1 = train.columns[train.nunique() == 1]
  binary_features = np.setdiff1d(temp, card1)
  print("%d features - %d features = %d binary features, %d categorical features" % (len(temp), len(card1), len(binary_features), len(categorical_features)))

  # feature engineering을 위해 tempX, y 생성
  total = pd.concat([train, test])
  split_point = len(train)
  total_OHE = pd.get_dummies(data=total, columns=categorical_features)
  y = train.운송장_건수
  tempX = total_OHE.drop(columns=["운송장_건수"])
  tempX = tempX.drop(columns=card1)
  trainX = tempX[:split_point]
  testX = tempX[split_point:]
  print(trainX.shape, testX.shape, y.shape)

  return y, trainX, testX

## 라벨인코딩

def label_encoder(train, test):

  from sklearn.preprocessing import LabelEncoder

  categorical_features = train.columns[0:12]
  temp = train.columns[13:]
  card1 = train.columns[train.nunique() == 1]
  binary_features = np.setdiff1d(temp, card1)
  print("%d features - %d features = %d binary features, %d categorical features" % (len(temp), len(card1), len(binary_features), len(categorical_features)))

  total = pd.concat([train, test])
  split_point = len(train)

  LE = LabelEncoder()

  total['물품_카테고리_소분류'] = LE.fit_transform(total['물품_카테고리_소분류'])
  total['물품_카테고리_대분류'] = LE.fit_transform(total['물품_카테고리_대분류'])
  total['송하인_코드1'] = LE.fit_transform(total['송하인_코드1'])
  total['송하인_코드2'] = LE.fit_transform(total['송하인_코드2'])
  total['송하인_코드3'] = LE.fit_transform(total['송하인_코드3'])
  total['송하인_코드4'] = LE.fit_transform(total['송하인_코드4'])
  total['송하인_코드5'] = LE.fit_transform(total['송하인_코드5'])
  total['수하인_코드1'] = LE.fit_transform(total['수하인_코드1'])
  total['수하인_코드2'] = LE.fit_transform(total['수하인_코드2'])
  total['수하인_코드3'] = LE.fit_transform(total['수하인_코드3'])
  total['수하인_코드4'] = LE.fit_transform(total['수하인_코드4'])
  total['수하인_코드5'] = LE.fit_transform(total['수하인_코드5'])

  y = train.운송장_건수
  tempX = total.drop(columns=["운송장_건수"])
  tempX = tempX.drop(columns=card1)
  trainX = tempX[:split_point]
  testX = tempX[split_point:]
  print(trainX.shape, testX.shape, y.shape)

  return y, trainX, testX

def none (train, test):

    categorical_features = train.columns[0:12]
    temp = train.columns[13:]
    card1 = train.columns[train.nunique() == 1]
    binary_features = np.setdiff1d(temp, card1)
    print("%d features - %d features = %d binary features, %d categorical features" % (len(temp), len(card1), len(binary_features, len(categorical_features))))
    
    total = pd.concat([train, test])
    split_point = len(train)
    y = train.운송장_건수
    tempX = total.drop(columns=["운송장_건수"])
    tempX = tempX.drop(columns=card1)
    trainX = tempX[:split_point]
    testX = tempX[split_point:]
    print(trainX.shape, testX.shape, y.shape)

    return y, trainX, testX

In [ ]:
# Feature Reducer

# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ#


#                     3-4. 차원축소                      #



# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ#


# 3-4. 차원축소

def feature_reducer(trainX, feature_reducing):

    if feature_reducing == "correlation":
        threshold = 0.7
        correlated_features = remove_collinearity(trainX, threshold)
        correlated_features = set(correlated_features) # 중복 제거
        print("%d Correlation features over %.2f" % (len(correlated_features), threshold))
        
        X = trainX.drop(columns=correlated_features)
        print(X.shape)
        return X, correlated_features
        
    elif feature_reducing == "feature_importance":
        show_plot = True
        model = RandomForestRegressor(max_features="sqrt", n_jobs=-1, random_state=0xC0FFEE)
        model.fit(trainX, y)
        important_features = find_feature_importance(trainX, model, show_plot)
        X = trainX[important_features]
        print(X.shape)
        return X, important_features
        
    elif feature_reducing == "PCA":
        show_plot = True
        pca_model, X = apply_PCA(trainX, show_plot)
        print(X.shape)
        return X, pca_model


## 1. correlation

# 중복정보가 있는 column 제거하기 위해 상관계수를 확인해봅니다.
def remove_collinearity(X, threshold):
    """
    X : feature matrix
    threshold : 다중공선성을 제거할 column을 고르는 기준 값. [0, 1]
    """
    
    corr = X.corr()
    candidate_cols = []
    
    for x in corr.iterrows():
        idx, row = x[0], x[1] # decoupling tuple
        # 해당 row는 이미 처리가 되어서 볼 필요가 없다.
        if idx in candidate_cols:
            continue
        #print(row[row > 0.7].index[1:])
        candidates = row[row > threshold].index[1:]

        # 자기 자신을 제외하고 threshold를 넘는 column이 있다면,
        if len(candidates) != 0:
            for col in candidates:
                candidate_cols.append(col)           
    
    return candidate_cols

def find_feature_importance(X, model, show_plot):

    feat_names = X.columns.values
    importances = model.feature_importances_
    std = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)
    indices = np.argsort(importances)[::-1][:20]

    plt.figure(figsize=(12,12))
    plt.title("Feature importances")
    plt.bar(range(len(indices)), importances[indices], color="r", align="center")
    plt.xticks(range(len(indices)), feat_names[indices], rotation='vertical')
    plt.xlim([-1, len(indices)])
    plt.show()
    
    important_features = X.columns[importances >= 0.005]
    return important_features
    
def apply_PCA(X, show_plot):
    from sklearn.decomposition import PCA
    # training data와 test data를 모두 PCA를 이용하여 차원 감소를 수행합니다.
    pca = PCA(n_components=0.90) # 원래 데이터의 90%를 보존하는 차원.
    pca_090 = pca.fit(X) # 학습 및 변환
    reduced_X = pca_090.transform(X)
    print(reduced_X.shape)
    
    if show_plot:
        labels = [f"PC{x}" for x in range(1, reduced_X.shape[1]+1)]
        pca_090_variance = np.round(pca_090.explained_variance_ratio_.cumsum()*100, decimals=1)
        plt.figure(figsize=(25,5))
        plt.bar(x=range(1, len(pca_090_variance)+1), height=pca_090_variance, tick_label=labels)

        plt.xticks(rotation=90, color='indigo', size=15)
        plt.yticks(rotation=0, color='indigo', size=15)
        plt.title('Scree Plot',color='tab:orange', fontsize=25)
        plt.xlabel('Principal Components', {'color': 'tab:orange', 'fontsize':15})
        plt.ylabel('Cumulative percentage of explained variance ', {'color': 'tab:orange', 'fontsize':15})
        plt.show()
        
        X_train_pca_df = pd.DataFrame(reduced_X, columns=labels)
        display(X_train_pca_df)

    return pca_090, X_train_pca_df

In [ ]:
# Modeling sets
# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ#


#                     4. Modeling                      #



# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ#



## 5가지 모델 세트

def model5(X_train, y_train, X_val, y_val):
  reg = LinearRegression()
  reg2 = Lasso()
  reg3 = Ridge()
  reg4 = XGBRegressor()
  reg5 = LGBMRegressor()

  reg.fit(X_train, y_train)
  reg2.fit(X_train, y_train)
  reg3.fit(X_train, y_train)
  reg4.fit(X_train, y_train)
  reg5.fit(X_train, y_train)

  pred_train = reg.predict(X_train)
  pred_train2 = reg2.predict(X_train)
  pred_train3 = reg3.predict(X_train)
  pred_train4 = reg4.predict(X_train)
  pred_train5 = reg5.predict(X_train)

  pred_val = reg.predict(X_val)
  pred_val2 = reg2.predict(X_val)
  pred_val3 = reg3.predict(X_val)
  pred_val4 = reg4.predict(X_val)
  pred_val5 = reg5.predict(X_val)

  mse_train = mean_squared_error(y_train, pred_train)
  mse_train2 = mean_squared_error(y_train, pred_train2)
  mse_train3 = mean_squared_error(y_train, pred_train3)
  mse_train4 = mean_squared_error(y_train, pred_train4)
  mse_train5 = mean_squared_error(y_train, pred_train5)

  mse_val = mean_squared_error(y_val, pred_val)
  mse_val2 = mean_squared_error(y_val, pred_val2)
  mse_val3 = mean_squared_error(y_val, pred_val3)
  mse_val4 = mean_squared_error(y_val, pred_val4)
  mse_val5 = mean_squared_error(y_val, pred_val5)

  r2_train = r2_score(y_train, pred_train)
  r2_train2 = r2_score(y_train, pred_train2)
  r2_train3 = r2_score(y_train, pred_train3)
  r2_train4 = r2_score(y_train, pred_train4)
  r2_train5 = r2_score(y_train, pred_train5)

  r2_val = r2_score(y_val, pred_val)
  r2_val2 = r2_score(y_val, pred_val2)
  r2_val3 = r2_score(y_val, pred_val3)
  r2_val4 = r2_score(y_val, pred_val4)
  r2_val5 = r2_score(y_val, pred_val5)

  print("------- train/val mse -------")
  print("1. Linear Regression train/val \t= %.4f, %.4f" % (mse_train, mse_val))
  print("2. Lasso, train/val \t\t = %.4f, %.4f" % (mse_train2, mse_val2))
  print("3. Ridge, train/val \t\t = %.4f, %.4f" % (mse_train3, mse_val3))
  print("4. XGBoost, train/val \t\t = %.4f, %.4f" % (mse_train4, mse_val4))
  print("5. LightGBM, train/val \t\t = %.4f, %.4f" % (mse_train5, mse_val5))

  print("------- train/val R2 score -------")
  print("1. Linear Regression train/val \t= %.4f, %.4f" % (r2_train, r2_val))
  print("2. Lasso, train/val \t\t = %.4f, %.4f" % (r2_train2, r2_val2))
  print("3. Ridge, train/val \t\t = %.4f, %.4f" % (r2_train3, r2_val3))
  print("4. XGBoost, train/val \t\t = %.4f, %.4f" % (r2_train4, r2_val4))
  print("5. LightGBM, train/val \t\t = %.4f, %.4f" % (r2_train5, r2_val5))

  return reg, reg2, reg3, reg4, reg5

In [ ]:
# Hyperparameter tuning
# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ#


#                     4. Modeling                      #



# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ#


## grid search

## grid serch CV 적용

def grid_search(X_train, y_train, regs):
  # Grid searchCV
  from sklearn.model_selection import GridSearchCV

  param_grid = {                            
    "max_depth": [1, 3, 5],
    "n_estimators": [100, 200, 300],
    "learning_rate": [0.1, 0.3, 0.5]
  }

  gcv = GridSearchCV(estimator=regs, param_grid=param_grid, scoring= 'neg_mean_squared_error', verbose = 1)
  gcv.fit(X_train, y_train)
  print(gcv.best_estimator_)
  print(gcv.best_params_)

  return gcv




## optuna
######




'''
옵튜나 코드 만들기
'''




#####


## prediction - 5models

def predict(gcv, X_train, y_train, X_val, X_test, y_test):
  # training set과 같은 전처리 : V
  final_model = gcv.best_estimator_
  final_model.fit(X_train, y_train)
  pred_train = final_model.predict(X_train)
  pred_val = final_model.predict(X_val)
  pred_test = final_model.predict(X_test)

    # PCA 적용
  if reducer == "correlation":
      test = testX.drop(columns=reduced)
      print(X.shape)
      
  elif reducer == "feature_importance":
      test = testX[reduced]
      print(X.shape)
      
  elif reducer == "PCA":
      test = reduced.transform(testX)
      print(X.shape)
      
  prediction = final_model.predict(test)

  mse_train = mean_squared_error(y_train, pred_train)
  mse_val = mean_squared_error(y_val, pred_val)
  mse_test = mean_squared_error(y_test, pred_test)

  r2_train = r2_score(y_train, pred_train)
  r2_val = r2_score(y_val, pred_val)
  r2_test = r2_score(y_test, pred_test)

  print("------- train/val mse -------")

  print("5. LightGBM, train/val/test \t\t = %.4f, %.4f, %.4f" % (mse_train, mse_val, mse_test) )

  print("------- train/val R2 score -------")

  print("5. LightGBM, train/val/test \t\t = %.4f, %.4f, %.4f" % (r2_train, r2_val, r2_test) )

  prediction = final_model.predict(test)
  
  submission["운송장_건수"] = prediction
  
  display(submission)

  return final_model, pred_train, pred_val, pred_test, prediction

# 2. 데이터 로딩

In [ ]:
# 뉴 데이터 로딩
test = pd.read_csv('data/final_test.csv', index_col = 0)             # unnamed_0 이라는 index가 추가되어 나오지 않게 'index_col = 0' 을 추가했습니다.
train = pd.read_csv('data/final_train.csv', index_col = 0)

submission = pd.read_csv('data/sample_submission.csv')

train.head()

# 3. 전처리

## 3-1. data cleansing

> 데이터에서 값이 잘못되거나 타입이 맞지 않는 행이나 열을 제거하는 작업을 수행합니다.

이 데이터에서는 결측치가 없으며, 데이터타입 또한 범주형으로 통일되어있습니다.

In [ ]:
# 필요시 추가하기

## 3-2. Feature construction

> 기존의 피쳐를 기반으로 새로운 피쳐를 만들어냅니다.

### 3-2-1. 고유번호 끊어주기
> 이 데이터에서는 격자공간 고유번호의 각 자리수가 단위별로 의미를 가지고 있는 것을 확인했습니다.
> 
> 16자리의 격자공간고유번호를 1~2 / 3~5 / 6~9 / 10 / 11~16 자리로 끊었습니다.

In [ ]:
train, test = setting_data(train, test, '0', '2', '5', '9', '10', '16')       # 0, 2, 5, 9, 10, 16번째 위치에서 코드를 잘라서 저장합니다. 이걸로 결정

In [ ]:
train.head()

### 3-2-2. 카테고리 대분류 추가

> 물품 카테고리를 좀 더 큰 항목으로 묶어, 100개의 소분류 카테고리를 10개의 대분류 카테고리로 분류했습니다.

In [ ]:
train, test = large_category(train, test)

In [ ]:
train.head()

# 원하는 학습 모델 선택

## 3-3. 인코딩 Representation transformation      << select_model 선택해줘야함!

> 원핫, 라벨, 민타겟 인코딩을 통해 데이터를 전처리하는 과정을 진행합니다.
> 
> lightGBM과 Catboost 모델은 인코딩을 하지 않고 돌리는게 성능이 더 좋습니다.

In [ ]:
# 원하는 모델 선택!
select_model = '5models' # '5models', 'catboost(미구현)', 'lightgbm(미구현)', 'randomforest(미구현)'

In [ ]:
# 선택한 모델에 따라 인코딩 선택!

if select_model == '5models':
  encoder_select = 'label_encoder' # 'label_encoder', 'one_hot_encoder'
elif select_model == 'catboost':
  encoder_select = 'none'
elif select_model == 'lightgbm':
  encoder_select = 'none'

In [ ]:
# 선택된 인코더 실행
if encoder_select == "label_encoder":
  y, trainX, testX = label_encoder(train, test)
  
elif encoder_select == "one_hot_encoder":
  y, trainX, testX = one_hot_encoder(train, test)

if encoder_select == "none":
  y, trainX, testX = none(train, test)

In [ ]:
trainX.head()

## 3-4. 차원 축소 Feature extraction        << reducer 선택해줘야함!

> 전체 feature의 수를 줄이거나 피쳐를 해시값으로 변환해 효율적인 피쳐를 사용하는 방법을 수행합니다.

In [ ]:
# 차원축소 방법 선택
reducer = "feature_importance" # "correlation" / "feature_importance" / "PCA"
X, reduced = feature_reducer(trainX, reducer) # "correlation" / "feature_importance" / "PCA" ,  X 는 차원축소된 데이터입니다.


In [ ]:
reducer

In [ ]:
reduced

## 3-4. Data partitioning
> 학습, 평가, 테스트용 데이터로 나누는 작업을 수행합니다.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape, X_val.shape, y_val.shape)

In [ ]:
# feature scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# 4. Modeling

- Model training

- Hyper-parameter tuning

- Evaluation

## 4-1. model training

In [ ]:
# 5models 모델 실행 
# 1 : linear regression, 
# 2 : ridge, 
# 3 : lasso, 
# 4 : XGBoost, 
# 5 : lightgbm

reg, reg2, reg3, reg4, reg5 = model5(X_train, y_train, X_val, y_val)

## 4-2. hyperparameter tuning  << select_reg 설정해줘야함!

In [ ]:
# grid search
select_reg = reg5                                     # reg = linear regression, reg2 = ridge, reg3 = lasso, reg4 = XGBoost, reg5 = lightgbm

gcv = grid_search(X_train, y_train, select_reg)       # regressor 모델 선택

## 4-3. evaluation

In [ ]:
final_model, pred_train, pred_val, pred_test, prediction = predict(gcv, X_train, y_train, X_val, X_test, y_test)

# 5. 제출파일 생성        << 극단값 조정해줘야함!
> submission 극단값 확인, 조정

In [ ]:
submission.loc[submission.운송장_건수>20,'운송장_건수']

In [ ]:
submission.loc[submission.운송장_건수>20,'운송장_건수']=submission.loc[submission.운송장_건수>20,'운송장_건수']*10

In [ ]:
# 파일명 : "제출시간 인코더_차원축소방법_(직접입력! 모델명).csv"

import datetime
path = "submissions/"
submission.reset_index(drop=True).to_csv(path + str(datetime.datetime.now()) + f"{encoder_select}_{reducer}_lgbm.csv", index=False)